<a href="https://colab.research.google.com/github/beyzakulakac/intern_project/blob/main/knn_algoritmasi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import itertools
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.ticker import NullFormatter
%matplotlib inline

In [ ]:
df = pd.read_csv("/content/teleCust1000t.csv")
df.head()

,region,tenure,age,marital,address,income,ed,employ,retire,gender,reside,custcat
0,2,13,44,1,9,64.0,4,5,0.0,0,2,1
1,3,11,33,1,7,136.0,5,5,0.0,0,6,4
2,3,68,52,1,24,116.0,1,29,0.0,1,2,3
3,2,33,33,0,12,33.0,2,0,0.0,1,1,1
4,2,23,30,1,9,30.0,1,2,0.0,0,4,3


In [ ]:
df['custcat'].value_counts()

,count
custcat,
3,281
1,266
4,236
2,217


In [ ]:
df.columns

Index(['region', 'tenure', 'age', 'marital', 'address', 'income', 'ed',
       'employ', 'retire', 'gender', 'reside', 'custcat'],
      dtype='object')

In [ ]:
x = df.drop('custcat', axis=1)
x[0:5]

,region,tenure,age,marital,address,income,ed,employ,retire,gender,reside
0,2,13,44,1,9,64.0,4,5,0.0,0,2
1,3,11,33,1,7,136.0,5,5,0.0,0,6
2,3,68,52,1,24,116.0,1,29,0.0,1,2
3,2,33,33,0,12,33.0,2,0,0.0,1,1
4,2,23,30,1,9,30.0,1,2,0.0,0,4


In [ ]:
y = df['custcat']
y[0:5]

,custcat
0,1
1,4
2,3
3,1
4,3


K-NN algortimasını kullanabilmek için verileri normalize ettik. Noktalar arası verilerin hesaplandığı algoritmalarda verileri normalize etmek önemlidir.

In [ ]:
x = preprocessing.StandardScaler().fit(x).transform(x)
x[0:5]

array([[-0.02696767, -1.055125  ,  0.18450456,  1.0100505 , -0.25303431,
        -0.12650641,  1.0877526 , -0.5941226 , -0.22207644, -1.03459817,
        -0.23065004],
       [ 1.19883553, -1.14880563, -0.69181243,  1.0100505 , -0.4514148 ,
         0.54644972,  1.9062271 , -0.5941226 , -0.22207644, -1.03459817,
         2.55666158],
       [ 1.19883553,  1.52109247,  0.82182601,  1.0100505 ,  1.23481934,
         0.35951747, -1.36767088,  1.78752803, -0.22207644,  0.96655883,
        -0.23065004],
       [-0.02696767, -0.11831864, -0.69181243, -0.9900495 ,  0.04453642,
        -0.41625141, -0.54919639, -1.09029981, -0.22207644,  0.96655883,
        -0.92747794],
       [-0.02696767, -0.58672182, -0.93080797,  1.0100505 , -0.25303431,
        -0.44429125, -1.36767088, -0.89182893, -0.22207644, -1.03459817,
         1.16300577]])

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state = 4)
print("train set: ", x_train.shape, y_train.shape)
print("test set: ", x_test.shape, y_test.shape)

train set:  (800, 11) (800,)
test set:  (200, 11) (200,)


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
k = 5
neigh = KNeighborsClassifier(n_neighbors=k).fit(x_train, y_train)
neigh

KNeighborsClassifier()

In [ ]:
y_hat = neigh.predict(x_test)
y_hat[0:5]

array([1, 3, 3, 4, 4])

In [ ]:
from sklearn import metrics
print("train seti doğruluğu: ", metrics.accuracy_score(y_train, neigh.predict(x_train)))
print("test seti doğruluğu: ", metrics.accuracy_score(y_test, y_hat ))

train seti doğruluğu:  0.5375
test seti doğruluğu:  0.315


üstteki kodun çıktısını gözlemlediğimizde k değerini ilk 4 olarak belirlediğimizde train test doğruluğu 0.54 olarak gözüktü, k değerini 1 er 1 er azalttığımda train test doğruluğu arttı ancak test seti dığruluğu giderek azaldı. Bu gözlemden şu sonuca ulaşabiliriz: k değerini azalttıkça hedef çembere yaklaşıyoruz bu durumda train doğruluğu artıyor ancak model overfit oluyor yani öğrenmek yerine ezberliyor. k değerini 4'ten yüksek seçtiğimizde de muhtemelen çok genel tahminler verecek. Aşağıdaki kodda doğru k sayısını bulmaya çalışacağız.

In [ ]:
Ks = 10 #deneyeceğimiz komşu sayılarının üst sınırı
mean_acc = np.zeros((Ks-1)) #her komşu sayısı için doğruluk oranlarını belirlemek üzere bir dizi oluşturur.
std_acc = np.zeros((Ks-1)) #her komşu sayısı için standart sapmaları belirlemek üzere bir dizi oluşturur.
ConfusionMx = []; #karışıklık matrislerini depolamak üzere boş bir liste oluşturur.
for n in range(1,Ks): #n sayısı yani komşu sayısı 1 ile ks yani 10 arasında dolanıyor.döngüye girdi.
  neigh = KNeighborsClassifier(n_neighbors=n).fit(x_train, y_train) #döngüdeki o anki n sayısına göre eğitim yapılır.
  y_hat = neigh.predict(x_test) #test veri kümesindeki örnekler için tahmin yapılır.
  mean_acc[n-1] = metrics.accuracy_score(y_test, y_hat) #gerçek etiketler ile test veri kümesi kullanılarsk yapılan tahminler aarsında doğruluk hesaplanır.
  std_acc[n-1] = np.std(y_hat==y_test)/np.sqrt(y_hat.shape[0]) #gerçek etiketler ile test veri kümesi kullanılarak tahmin edilen etiketler arasındaki standart sapma hesaplanır.
mean_acc #komşu sayısı olan n değeri, doğruluğun kaydedildiği mean_acc dizisi ve standart sapmaların kaydedildiği std_acc dizisi çıktı olarak döndürülür.

array([0.3  , 0.29 , 0.315, 0.32 , 0.315, 0.31 , 0.335, 0.325, 0.34 ])

yukarıdaki kodun çıktısına bakıldığında dizinin içindeki değerler sırasıyla k değerlerine göre doğruluk oranları. karşılaştırma yapıldığında k=9 değerinde en yüksek doğruluğu verdiğini görüyoruz.